In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

## Using kernel
### https://www.kaggle.com/dster/two-sigma-news-official-getting-started-kernel
### kernel에 이렇게 노트북을 쓸 수 있는 공간이 있다는 것을 처음 알게되었습니다.
### 위의 대회에서 kernel을 꼭 써야하는 규칙이 있습니다.
###  2007년 2월 부터 2016년12월까지의 market,news데이터를 주는데 위 기간의 거래일에 대한 예측을하기 위해 미래의 데이터를 사용하지 않도록 정보 흐름을 제어하기 위해 이용해야 한다고 합니다.

----

# End to End usage Example

In [ ]:
 from kaggle.competitions import twosigmanews
 env = twosigmanews.make_env()

(market_train_df, news_train_df) = env.get_training_data()
train_my_model(market_train_df, news_train_df)

for (market_obs_df, news_obs_df, predictions_template_df) in env.get_prediction_days():
  predictions_df = make_my_predictions(market_obs_df, news_obs_df, predictions_template_df)
  env.predict(predictions_df)

env.write_submission_file()

-----

## 1. Get Env&data

In [ ]:
# get environment!

from kaggle.competitions import twosigmanews
# You can only call make_env() once, so don't lose it!
env = twosigmanews.make_env()

## get_training_data function
## trainging data을 튜플로 return
### - market_train_df: DataFrame with market training data
### - news_train_df: DataFrame with news training data


In [ ]:
(market_train_df, news_train_df) = env.get_training_data()

In [ ]:
# market data
market_train_df.head()

In [ ]:
# news data
news_train_df.head()

### get_prediction_days function
### train data 이후의 test data
### market_observations_df:  market test data
### news_observations_df: news test data
### predictions_template_df: DataFrame with assetCode and confidenceValue columns, prefilled with confidenceValue = 0, to be filled in and passed back to the predict function. 제출용 템플릿

In [ ]:
# You can only iterate through a result from `get_prediction_days()` once
# so be careful not to lose it once you start iterating.
days = env.get_prediction_days()

In [ ]:
(market_obs_df, news_obs_df, predictions_template_df) = next(days)


In [ ]:
#market test data
market_obs_df.head()

In [ ]:
#news test data
news_obs_df.head()

In [ ]:
#prediction template
predictions_template_df.head()

In [ ]:
import pandas

In [ ]:
news_train_df.describe()

In [ ]:
market_train_df.describe()

In [ ]:
market_train_df.head(15)

In [ ]:
market_train_df.groupby('assetName').describe()

Market data

    - 이 데이터에는 미국 상장된 기관이 포함됩니다. 포함된 상장된 기관은  매일 변경되며 거래 금액과 정보의 가용성에 따라 결정됩니다. 즉, 이 데이터 서브셋으로 들어가 그대로 두는 계측기가 있을 수 있습니다. 따라서 제공된 데이터에 차이가 있을 수 있으며, 이는 데이터가 존재하지 않는다는 것을 반드시 의미하지는 않습니다(선택 기준으로 인해 행에 포함되지 않을 가능성이 있음).

The marketdata contains a variety of returns calculated over different timespans. All of the returns in this set of marketdata have these properties:

수익(Returns)

    - 수익은 항상 개장에서 개장(한 거래일의 개시 시간부터 다른 거래일의 개방 시점까지) 또는 폐장에서 폐장(거래일 마감 시간부터 다른 거래일의 개방 시점까지)로 계산됩니다.

    - 수익은 데이터가 벤치마크에 따라 조정되지 않거나 시장 재평가(Mktres)되지 않는다는 뜻이며, 이는 시장의 움직임이 전체적으로 고려되었음을 의미하며 계측기에 고유한 움직임만 남깁니다.

    - 수익은 임의의 간격에 걸쳐 계산할 수 있습니다. 여기에 1일 과10일 지평선이 제공됩니다.

    - 수익에는 과거와 비교해 보는 경우에는 'Prev'라는 태그가 붙고, 미래와 비교해 보는 경우에는 'Next'라는 태그가 붙습니다.

- time(datetime64[ns, UTC]) 
    - the current time (in marketdata, all rows are taken at 22:00 UTC) 
    - 현재 시각 (marketdata에서는 모두 22시)

- assetCode(object) 
    - a unique id of an asset
    - asset의 unique id(primary key로 이용하자!!)

- assetName(category) 
    - the name that corresponds to a group of assetCodes. 
      These may be "Unknown" if the corresponding assetCode does not have any rows in the news data.
    - 자산 코드 그룹에 해당하는 이름. 
        해당 AssetCode에 맞는 뉴스 데이터에 행이 없는 경우 이러한 행은 "알 수 없음"으로 표시됨

- universe(float64)
    - a boolean indicating whether or not the instrument on that day will be included in scoring. This value is not provided outside of the training data time period. The trading universe on a given date is the set of instruments that are avilable for trading (the scoring function will not consider instruments that are not in the trading universe). The trading universe changes daily.
    - 그날 이 기구가 스코어링에 들어가는지 안하는지에 대한 불리언 값, 트레이닝데이터 시간 밖의 값은 제공하지 않음

- volume(float64) 
    - trading volume in shares for the day
    - 당일 주식 거래량

- close(float64)
    - the close price for the day (not adjusted for splits or dividends)
    - 당일의 마감 가격(분할 또는 배당금에 대해 조정되지 않음)

- open(float64) 
    - the open price for the day (not adjusted for splits or dividends)
    - 당일의 공개 가격(분할 또는 배당금에 대해 조정되지 않음)

- returnsClosePrevRaw1(float64) and returnsClosePrevRaw10(float64) 
    - (오늘의 폐장 가격 - 하루 전(10일전) 폐장 가격)/하루 전 폐장 가격

- returnsOpenPrevRaw1(float64) and returnsOpenPrevRaw10(float64)
    - (오늘의 개장 가격 - 하루 전 개장 가격)/하루 전 개장 가격

- returnsClosePrevMktres1(float64) and returnsClosePrevMktres10(float64)
    - (오늘의 폐장 Mktres - 하루 전(10일전) 폐장 Mktres)/하루 전 폐장 Mktres
- returnsOpenPrevMktres1(float64) and returnsOpenPrevMktres10(float64)
    - (오늘의 개장 Mktres - 하루 전(10일전) 개장 Mktres)/하루 전 개장 Mktres

- returnsOpenNextMktres10(float64)
    - 10일 후의 개장 Mktres - 오늘의 개장 Mktres
    - 이번 competiton의 scoring data
    - 마켓데이터에서 이 값은 null값이 없다


News data
The news data contains information at both the news article level and asset level (in other words, the table is intentionally not normalized).

time(datetime64[ns, UTC])
    - 데이터가 피드에 보여진 시간

sourceTimestamp(datetime64[ns, UTC])
    - 뉴스가 만들어진 시각

firstCreated(datetime64[ns, UTC]) 
    - 첫번쩨 버전이 나온 시간

sourceId(object) 
    - 각 뉴스 아이템의 id

headline(object) 
    - the item's headline

urgency(int8) 
    - differentiates story types (1: alert, 3: article)
    - 뉴스의 종류? 

takeSequence(int16) 
    - 뉴스의 순서
    - For a given story, alerts and articles have separate sequences.

provider(category) 
    - identifier for the organization which provided the news item (e.g. RTRS for Reuters News, BSW for Business Wire)
    - 뉴스 제공자
    
subjects(category) 
    - topic codes and company identifiers that relate to this news item. Topic codes describe the news item's subject matter.
    - These can cover asset classes, geographies, events, industries/sectors, and other types.

audiences(category) 
    - identifies which desktop news product(s) the news item belongs to. They are typically tailored to specific audiences. (e.g. "M" for Money International News Service and "FB" for French General News Service)
    
bodySize(int32) 
    - the size of the current version of the story body in characters
    
companyCount(int8) 
    - the number of companies explicitly listed in the news item in the subjects field
    
headlineTag(object) 
    - the Thomson Reuters headline tag for the news item
    
marketCommentary(bool)
    - boolean indicator that the item is discussing general market conditions, such as "After the Bell" summaries
    
sentenceCount(int16)
    - the total number of sentences in the news item. Can be used in conjunction with firstMentionSentence to determine the relative position of the first mention in the item.
    
wordCount(int32)
    - the total number of lexical tokens (words and punctuation) in the news item
assetCodes(category)
    - list of assets mentioned in the item
    
assetName(category)
    - name of the asset
    
firstMentionSentence(int16)
    - the first sentence, starting with the headline, in which the scored asset is mentioned.
    1: headline
    2: first sentence of the story body
    3: second sentence of the body, etc
    0: the asset being scored was not found in the news item's headline or body text. As a result, the entire news item's text (headline + body) will be used to determine the sentiment score.
    
relevance(float32) 
    - a decimal number indicating the relevance of the news item to the asset. It ranges from 0 to 1. If the asset is mentioned in the headline, the relevance is set to 1. When the item is an alert (urgency == 1), relevance should be gauged by firstMentionSentence instead.
    
sentimentClass(int8) 
    - indicates the predominant sentiment class for this news item with respect to the asset. The indicated class is the one with the highest probability.
sentimentNegative(float32) 
    - probability that the sentiment of the news item was negative for the asset
sentimentNeutral(float32) 
    - probability that the sentiment of the news item was neutral for the asset
sentimentPositive(float32) 
    - probability that the sentiment of the news item was positive for the asset
sentimentWordCount(int32) 
    - the number of lexical tokens in the sections of the item text that are deemed relevant to the asset. This can be used in conjunction with wordCount to determine the proportion of the news item discussing the asset.
    
noveltyCount12H(int16)
    - The 12 hour novelty of the content within a news item on a particular asset. It is calculated by comparing it with the asset-specific text over a cache of previous news items that contain the asset.
noveltyCount24H(int16) 
    - same as above, but for 24 hours
noveltyCount3D(int16) 
    - same as above, but for 3 days
noveltyCount5D(int16) 
    - same as above, but for 5 days
noveltyCount7D(int16) 
    - same as above, but for 7 days
    
volumeCounts12H(int16)
    - the 12 hour volume of news for each asset. A cache of previous news items is maintained and the number of news items that mention the asset within each of five historical periods is calculated.
volumeCounts24H(int16) 
    - same as above, but for 24 hours
volumeCounts3D(int16) 
    - same as above, but for 3 days
volumeCounts5D(int16) 
    - same as above, but for 5 days
volumeCounts7D(int16) 
    - same as above, but for 7 days